#**Loading the Dataset**

In [1]:
!pip install psynlig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.9/455.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: j

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from psynlig import (
    pca_explained_variance,
    pca_explained_variance_bar,
)
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [3]:
# Read the CSV for listings
listing_data = pd.read_csv('https://drive.google.com/uc?export=download&id=1mn47Wp7mbaiXxWjIDw3xQR3k1udXtxxu')

In [4]:
listing_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle Sanctuary,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,240.0,30,49,2022-06-21,0.27,3,365,0,NaN
1,6848,Only 2 stops to Manhattan studio,15991,Allen & Irina,Brooklyn,Williamsburg,40.70935,-73.95342,Entire home/apt,81.0,30,195,2024-10-05,1.03,1,196,4,NaN
2,6872,Uptown Sanctuary w/ Private Bath (Month to Month),16104,Kahshanna,Manhattan,East Harlem,40.80107,-73.94255,Private room,65.0,30,1,2022-06-05,0.03,2,83,0,NaN
3,6990,UES Beautiful Blue Room,16800,Cyn,Manhattan,East Harlem,40.78778,-73.94759,Private room,70.0,30,251,2024-12-01,1.36,1,8,5,NaN
4,7064,"Amazing location! Wburg. Large, bright & tranquil",17297,Joelle,Brooklyn,Williamsburg,40.71248,-73.95881,Private room,NaN,30,13,2022-09-12,0.07,2,0,0,NaN


#**Listing EDA and Preprocessing**

In [5]:
listing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37784 entries, 0 to 37783
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              37784 non-null  int64  
 1   name                            37782 non-null  object 
 2   host_id                         37784 non-null  int64  
 3   host_name                       37778 non-null  object 
 4   neighbourhood_group             37784 non-null  object 
 5   neighbourhood                   37784 non-null  object 
 6   latitude                        37784 non-null  float64
 7   longitude                       37784 non-null  float64
 8   room_type                       37784 non-null  object 
 9   price                           22969 non-null  float64
 10  minimum_nights                  37784 non-null  int64  
 11  number_of_reviews               37784 non-null  int64  
 12  last_review                     

In [6]:
listing_data.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
count,3.778400e+04,3.778400e+04,37784.000000,37784.000000,22969.000000,37784.000000,37784.000000,25892.000000,37784.000000,37784.000000,37784.000000
mean,4.132488e+17,1.698767e+08,40.728805,-73.947311,195.224128,28.882172,25.658639,0.866954,71.636354,163.400963,3.731447
std,4.911855e+17,1.850207e+08,0.056120,0.054543,353.251037,29.905150,62.619846,1.885964,224.585038,148.521232,18.798642
min,2.595000e+03,1.678000e+03,40.500366,-74.251907,8.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000
25%,2.132202e+07,1.747741e+07,40.688662,-73.983316,82.000000,30.000000,0.000000,0.090000,1.000000,0.000000,0.000000
50%,4.998368e+07,8.703937e+07,40.726379,-73.954930,132.000000,30.000000,3.000000,0.290000,2.000000,155.000000,0.000000
75%,8.897044e+17,3.052402e+08,40.762310,-73.928196,223.000000,30.000000,22.000000,1.000000,9.000000,329.000000,1.000000
max,1.325354e+18,6.691812e+08,40.911390,-73.713650,20000.000000,1250.000000,2485.000000,116.300000,1154.000000,365.000000,1779.000000


In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

test_listing = listing_data.drop(['host_id', 'id', 'name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'last_review', 'license'], axis=1)

# Identify categorical and numerical columns
categorical_cols = test_listing.select_dtypes(include=['object']).columns
numerical_cols = test_listing.select_dtypes(include=['float64', 'int64']).columns

# Define transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols),
        ('num', 'passthrough', numerical_cols)
    ]
)

# Apply transformation
transformed_data = preprocessor.fit_transform(test_listing)

# Get feature names
encoded_cat_columns = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
final_columns = list(encoded_cat_columns) + list(numerical_cols)

# Convert back to DataFrame
listings_encoded = pd.DataFrame(transformed_data, columns=final_columns)
listings_encoded.head()

,room_type_Hotel room,room_type_Private room,room_type_Shared room,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
0,0.0,0.0,0.0,40.75356,-73.98559,240.0,30.0,49.0,0.27,3.0,365.0,0.0
1,0.0,0.0,0.0,40.70935,-73.95342,81.0,30.0,195.0,1.03,1.0,196.0,4.0
2,0.0,1.0,0.0,40.80107,-73.94255,65.0,30.0,1.0,0.03,2.0,83.0,0.0
3,0.0,1.0,0.0,40.78778,-73.94759,70.0,30.0,251.0,1.36,1.0,8.0,5.0
4,0.0,1.0,0.0,40.71248,-73.95881,NaN,30.0,13.0,0.07,2.0,0.0,0.0


In [8]:
from sklearn.linear_model import LinearRegression

regression_data = listings_encoded.dropna(subset=['price'])

# Create features and target
x = regression_data[['latitude', 'longitude', 'room_type_Hotel room','room_type_Private room','room_type_Shared room']]
y = regression_data['price']

# Fit linear regression model
model = LinearRegression()
model.fit(x, y)

# Create a new column with our predictions
listings_encoded.loc[:, 'price_predicted'] = model.predict(listings_encoded.loc[:, ['latitude', 'longitude', 'room_type_Hotel room','room_type_Private room','room_type_Shared room']])

listing_data['price'] = listing_data['price'].fillna(listings_encoded['price_predicted'])
listing_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37784 entries, 0 to 37783
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              37784 non-null  int64  
 1   name                            37782 non-null  object 
 2   host_id                         37784 non-null  int64  
 3   host_name                       37778 non-null  object 
 4   neighbourhood_group             37784 non-null  object 
 5   neighbourhood                   37784 non-null  object 
 6   latitude                        37784 non-null  float64
 7   longitude                       37784 non-null  float64
 8   room_type                       37784 non-null  object 
 9   price                           37784 non-null  float64
 10  minimum_nights                  37784 non-null  int64  
 11  number_of_reviews               37784 non-null  int64  
 12  last_review                     

#**Association Rules**

In [9]:
df_rules = listing_data.copy()

In [10]:
df_rules.drop(columns=['last_review', 'reviews_per_month', 'license'], inplace=True)

In [11]:
df_rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37784 entries, 0 to 37783
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              37784 non-null  int64  
 1   name                            37782 non-null  object 
 2   host_id                         37784 non-null  int64  
 3   host_name                       37778 non-null  object 
 4   neighbourhood_group             37784 non-null  object 
 5   neighbourhood                   37784 non-null  object 
 6   latitude                        37784 non-null  float64
 7   longitude                       37784 non-null  float64
 8   room_type                       37784 non-null  object 
 9   price                           37784 non-null  float64
 10  minimum_nights                  37784 non-null  int64  
 11  number_of_reviews               37784 non-null  int64  
 12  calculated_host_listings_count  

In [12]:
# Categorized Price into 3 based on Quantiles
df_rules['price_category'] = pd.qcut(df_rules['price'], q=3, labels=['Cheap', 'Moderate', 'Expensive'])

# Categorized the number of listings of the host; with 5 as a threshold for a moderate number of listings
df_rules['host_listings_count'] = pd.cut(
    df_rules['calculated_host_listings_count'],
    bins=[0, 1, 5, df_rules['calculated_host_listings_count'].max()], labels=['One', 'Some', 'Many'], include_lowest=True
)

# Categorized the number of reviews for each listing; with 10 as a threshold for a moderate number of reviews, and 50 for high number of reviews
df_rules['review_count'] = pd.cut(
    df_rules['number_of_reviews'],
    bins=[-0.1, 0, 10, 50, df_rules['number_of_reviews'].max()], labels=['None', 'Few', 'Average', 'High'], include_lowest=True
)

# Categorized the minimum number of nights for each listing; setting the threshold at 30 based on the requirements of Local Law 18
df_rules['night_count'] = pd.cut(
    df_rules['minimum_nights'],
    bins=[0, 29, df_rules['minimum_nights'].max()], labels=['Short', 'Long'], include_lowest=True
)

In [13]:
def association_analysis(subset, measurement="confidence"):
  listings = subset.apply(lambda x: list(x.dropna().astype(str)), axis=1).tolist()

  te = TransactionEncoder()
  transactions = te.fit_transform(listings)

  df_encoded = pd.DataFrame(transactions, columns=te.columns_)

  frequent_itemsets = apriori(df_encoded, min_support=0.00001, use_colnames=True)

  rules = association_rules(frequent_itemsets,
                          num_itemsets=frequent_itemsets.shape[0],
                          metric="support", min_threshold=0.05) #, metric="confidence", min_threshold=0.6
  rules = rules.sort_values(by=measurement, ascending=False)
  return rules

##**All Columns**

In [14]:
neighborhood_room_price = df_rules[['neighbourhood_group', 'room_type', 'price_category', 'review_count', 'host_listings_count', 'night_count']]
general = association_analysis(neighborhood_room_price, "confidence")

In [15]:
general.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
1115,"(None, One, Brooklyn)",(Long),0.051053,0.861979,0.050709,0.993261,1.152303,1.0,0.006702,20.480250,0.139284,0.058805,0.951172,0.526045
1857,"(Many, None, Cheap, Private room)",(Long),0.051053,0.861979,0.050683,0.992742,1.151702,1.0,0.006676,19.017375,0.138806,0.058773,0.947417,0.525770
1534,"(Entire home/apt, None, One)",(Long),0.067833,0.861979,0.067171,0.990246,1.148806,1.0,0.008701,14.149952,0.138956,0.077867,0.929328,0.534086
1199,"(None, Cheap, Private room)",(Long),0.087762,0.861979,0.086730,0.988239,1.146477,1.0,0.011081,11.735357,0.140054,0.100497,0.914787,0.544428
1917,"(Long, Few, One, Expensive)",(Entire home/apt),0.084507,0.533559,0.083501,0.988099,1.851901,1.0,0.038412,39.193304,0.502477,0.156204,0.974485,0.572298
1016,"(Entire home/apt, Few, Brooklyn)",(Long),0.064101,0.861979,0.063307,0.987614,1.145752,1.0,0.008053,11.142926,0.135923,0.073376,0.910257,0.530529
563,"(Entire home/apt, None)",(Long),0.153187,0.861979,0.151149,0.986697,1.144688,1.0,0.019105,10.374906,0.149265,0.174937,0.903614,0.581024
1505,"(Entire home/apt, Many, None)",(Long),0.067542,0.861979,0.066642,0.986677,1.144665,1.0,0.008422,10.359723,0.135537,0.077232,0.903472,0.531995
2037,"(Entire home/apt, Many, None, Manhattan)",(Long),0.059258,0.861979,0.058464,0.986601,1.144577,1.0,0.007385,10.300996,0.134272,0.067763,0.902922,0.527213
852,"(None, One)",(Long),0.130082,0.861979,0.128308,0.986368,1.144307,1.0,0.016181,10.125002,0.144966,0.148548,0.901235,0.567611


##**Minimum nights to Price or Location**

In [16]:
neighborhood_room_price = df_rules[['night_count','neighbourhood_group', 'price_category']]
table1 = association_analysis(neighborhood_room_price, "confidence")

In [17]:
table1.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
32,"(Cheap, Brooklyn)",(Long),0.133919,0.861979,0.121824,0.909684,1.055344,1.0,0.006389,1.528202,0.060550,0.139375,0.345636,0.525507
37,"(Brooklyn, Expensive)",(Long),0.108273,0.861979,0.098454,0.909313,1.054914,1.0,0.005125,1.521955,0.058376,0.112933,0.342951,0.511766
9,(Cheap),(Long),0.334242,0.861979,0.299677,0.896587,1.040150,1.0,0.011568,1.334665,0.057980,0.334258,0.250748,0.622125
56,"(Cheap, Queens)",(Long),0.090488,0.861979,0.080854,0.893536,1.036611,1.0,0.002856,1.296415,0.038831,0.092764,0.228642,0.493668
5,(Brooklyn),(Long),0.366160,0.861979,0.326964,0.892953,1.035934,1.0,0.011341,1.289349,0.054726,0.362819,0.224415,0.636135


##**Host Listing to Room Type, Price, Location?**

In [18]:
neighborhood_room_price = df_rules[['host_listings_count','room_type', 'price_category', 'neighbourhood_group']]
table2 = association_analysis(neighborhood_room_price, "confidence")

In [19]:
table2.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
271,"(Manhattan, One, Expensive)",(Entire home/apt),0.105468,0.533559,0.102424,0.971142,1.820120,1.0,0.046151,16.163188,0.503711,0.190875,0.938131,0.581553
170,"(One, Expensive)",(Entire home/apt),0.202361,0.533559,0.194103,0.959194,1.797728,1.0,0.086132,11.430790,0.556320,0.358245,0.912517,0.661492
243,"(Brooklyn, One, Expensive)",(Entire home/apt),0.082522,0.533559,0.079002,0.957344,1.794261,1.0,0.034971,10.935056,0.482483,0.147095,0.908551,0.552705
92,"(Brooklyn, Expensive)",(Entire home/apt),0.108273,0.533559,0.101948,0.941579,1.764713,1.0,0.044178,7.984140,0.485951,0.188833,0.874752,0.566325
31,(Expensive),(Entire home/apt),0.332892,0.533559,0.300498,0.902687,1.691822,1.0,0.122880,4.793213,0.612976,0.530958,0.791372,0.732941
158,"(Manhattan, Expensive)",(Entire home/apt),0.202334,0.533559,0.179071,0.885023,1.658716,1.0,0.071113,4.056815,0.497858,0.321593,0.753501,0.610319
199,"(Many, Expensive)",(Manhattan),0.086201,0.445136,0.074476,0.863985,1.940949,1.0,0.036105,4.079444,0.530519,0.163017,0.754869,0.515648
255,"(Entire home/apt, Many, Expensive)",(Manhattan),0.066827,0.445136,0.057299,0.857426,1.926213,1.0,0.027552,3.891758,0.515281,0.126026,0.743047,0.493075
144,"(Cheap, Queens)",(Private room),0.090488,0.448126,0.076726,0.847909,1.892120,1.0,0.036176,3.628570,0.518402,0.166113,0.724409,0.509562
174,"(Entire home/apt, Many)",(Manhattan),0.143817,0.445136,0.120977,0.841185,1.889728,1.0,0.056959,3.493782,0.549910,0.258511,0.713777,0.556481


##**Neighborhood and Room Leading to Price**

In [20]:
neighborhood_room_price = df_rules[['neighbourhood_group','room_type', 'price_category']]
table3 = association_analysis(neighborhood_room_price, "confidence")

In [21]:
table3.head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
44,"(Brooklyn, Expensive)",(Entire home/apt),0.108273,0.533559,0.101948,0.941579,1.764713,1.0,0.044178,7.984140,0.485951,0.188833,0.874752,0.566325
19,(Expensive),(Entire home/apt),0.332892,0.533559,0.300498,0.902687,1.691822,1.0,0.122880,4.793213,0.612976,0.530958,0.791372,0.732941
68,"(Manhattan, Expensive)",(Entire home/apt),0.202334,0.533559,0.179071,0.885023,1.658716,1.0,0.071113,4.056815,0.497858,0.321593,0.753501,0.610319
60,"(Cheap, Queens)",(Private room),0.090488,0.448126,0.076726,0.847909,1.892120,1.0,0.036176,3.628570,0.518402,0.166113,0.724409,0.509562
62,"(Queens, Private room)",(Cheap),0.091838,0.334242,0.076726,0.835447,2.499526,1.0,0.046030,4.045850,0.660591,0.219621,0.752833,0.532499
37,"(Cheap, Brooklyn)",(Private room),0.133919,0.448126,0.106871,0.798024,1.780801,1.0,0.046858,2.732369,0.506252,0.224908,0.634017,0.518254
14,(Cheap),(Private room),0.334242,0.448126,0.249815,0.747407,1.667849,1.0,0.100032,2.184832,0.601458,0.469089,0.542299,0.652436
21,(Manhattan),(Entire home/apt),0.445136,0.533559,0.282553,0.634758,1.189668,1.0,0.045047,1.277074,0.287330,0.405885,0.216960,0.582161
66,"(Entire home/apt, Manhattan)",(Expensive),0.282553,0.332892,0.179071,0.633758,1.903793,1.0,0.085011,1.821494,0.661698,0.410359,0.451000,0.585841
34,(Queens),(Private room),0.147020,0.448126,0.091838,0.624662,1.393943,1.0,0.025954,1.470340,0.331322,0.182468,0.319885,0.414800


##**Breakdown of Association Rules by Neighborhood**

In [22]:
neighborhood_deeper = df_rules[['neighbourhood_group', 'neighbourhood', 'room_type', 'price_category', 'review_count', 'host_listings_count']]

# Where our different borough results will be stored, set of Dataframes
borough_association_results = {}

# Run association analysis for each borough
for borough, subset in neighborhood_deeper.groupby('neighbourhood_group'):
    borough_association_results[borough] = association_analysis(subset.drop(columns=['neighbourhood_group']), "confidence")

In [23]:
borough_association_results['Manhattan'].head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
245,"(Few, One, Expensive)",(Entire home/apt),0.099411,0.634758,0.097747,0.983254,1.549020,1.0,0.034644,21.810147,0.393555,0.153587,0.954150,0.568622
142,"(One, Expensive)",(Entire home/apt),0.236934,0.634758,0.230097,0.971142,1.529939,1.0,0.079701,12.656419,0.453931,0.358632,0.920989,0.666819
273,"(None, One, Expensive)",(Entire home/apt),0.069386,0.634758,0.066948,0.964867,1.520054,1.0,0.022905,10.396026,0.367638,0.105067,0.903809,0.535169
124,"(Few, Expensive)",(Entire home/apt),0.153755,0.634758,0.140199,0.911833,1.436504,1.0,0.042602,4.142610,0.359075,0.216251,0.758606,0.566351
100,"(Average, Expensive)",(Entire home/apt),0.073429,0.634758,0.066770,0.909312,1.432532,1.0,0.020160,4.027442,0.325863,0.104097,0.751703,0.507250
23,(Expensive),(Entire home/apt),0.454545,0.634758,0.402283,0.885023,1.394268,1.0,0.113757,3.176647,0.518425,0.585547,0.685203,0.759390
136,"(None, Expensive)",(Entire home/apt),0.195553,0.634758,0.167727,0.857708,1.351235,1.0,0.043598,2.566837,0.323124,0.253141,0.610416,0.560973
270,"(Entire home/apt, None, One)",(Expensive),0.078126,0.454545,0.066948,0.856925,1.885236,1.0,0.031436,3.812379,0.509356,0.143751,0.737697,0.502106
242,"(Entire home/apt, Few, One)",(Expensive),0.114454,0.454545,0.097747,0.854026,1.878857,1.0,0.045722,3.736655,0.528218,0.207419,0.732381,0.534534
43,(Upper East Side),(Entire home/apt),0.089542,0.634758,0.074796,0.835325,1.315974,1.0,0.017959,2.217960,0.263720,0.115159,0.549135,0.476580


In [23]:
borough_association_results['Brooklyn'].head(20)

In [24]:
borough_association_results['Queens'].head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
290,"(Many, None, Cheap)",(Private room),0.093069,0.624662,0.091089,0.978723,1.566804,1.0,0.032952,17.640864,0.398882,0.145361,0.943313,0.562272
211,"(Woodside, Cheap)",(Private room),0.058506,0.624662,0.056346,0.963077,1.541756,1.0,0.019799,10.165392,0.373225,0.089891,0.901627,0.526639
210,"(Woodside, Private room)",(Cheap),0.058866,0.615482,0.056346,0.957187,1.555183,1.0,0.020115,8.981252,0.379318,0.091174,0.888657,0.524367
164,"(Private room, Flushing)",(Cheap),0.056526,0.615482,0.053645,0.949045,1.541955,1.0,0.018855,7.546175,0.372530,0.086754,0.867483,0.518102
180,"(Many, Cheap)",(Private room),0.203240,0.624662,0.192619,0.947741,1.517206,1.0,0.065663,7.182306,0.427850,0.303202,0.860769,0.628049
276,"(Many, Few, Cheap)",(Private room),0.073627,0.624662,0.067867,0.921760,1.475614,1.0,0.021875,4.797283,0.347933,0.107653,0.791549,0.515203
192,"(None, Cheap)",(Private room),0.187399,0.624662,0.171557,0.915466,1.465537,1.0,0.054496,4.440072,0.390913,0.267847,0.774778,0.595053
291,"(Many, None, Private room)",(Cheap),0.099730,0.615482,0.091089,0.913357,1.483972,1.0,0.029707,4.437984,0.362261,0.145948,0.774672,0.530677
277,"(Many, Few, Private room)",(Cheap),0.074887,0.615482,0.067867,0.906250,1.472424,1.0,0.021775,4.101530,0.346820,0.109023,0.756189,0.508258
181,"(Many, Private room)",(Cheap),0.212601,0.615482,0.192619,0.906012,1.472037,1.0,0.061767,4.091138,0.407252,0.303116,0.755569,0.609484


In [25]:
borough_association_results['Staten Island'].head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
51,(Expensive),(One),0.149864,0.566757,0.130790,0.872727,1.539860,1.0,0.045854,3.404048,0.412393,0.223256,0.706232,0.551748
209,"(Entire home/apt, Expensive)",(One),0.119891,0.566757,0.103542,0.863636,1.523820,1.0,0.035593,3.177112,0.390582,0.177570,0.685249,0.523164
35,(Expensive),(Entire home/apt),0.149864,0.495913,0.119891,0.800000,1.613187,1.0,0.045572,2.520436,0.447115,0.227979,0.603243,0.520879
33,(West Brighton),(Cheap),0.065395,0.558583,0.051771,0.791667,1.417276,1.0,0.015243,2.118801,0.315022,0.090476,0.528035,0.442175
210,"(One, Expensive)",(Entire home/apt),0.130790,0.495913,0.103542,0.791667,1.596383,1.0,0.038682,2.419619,0.429797,0.197917,0.586712,0.500229
72,(Mariners Harbor),(One),0.076294,0.566757,0.059946,0.785714,1.386332,1.0,0.016705,2.021798,0.301689,0.102804,0.505391,0.445742
167,"(Few, Some)",(Cheap),0.084469,0.558583,0.065395,0.774194,1.385995,1.0,0.018212,1.954846,0.304191,0.113208,0.488451,0.445633
106,"(Average, Private room)",(Cheap),0.141689,0.558583,0.108992,0.769231,1.377111,1.0,0.029847,1.912807,0.319048,0.184332,0.477208,0.482176
250,"(Moderate, None)",(One),0.070845,0.566757,0.054496,0.769231,1.357249,1.0,0.014344,1.877384,0.283284,0.093458,0.467344,0.432692
232,"(Entire home/apt, None)",(One),0.081744,0.566757,0.062670,0.766667,1.352724,1.0,0.016341,1.856754,0.283963,0.106977,0.461426,0.438622


In [26]:
borough_association_results['Bronx'].head(20)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
165,"(Many, Private room)",(Cheap),0.078642,0.635762,0.073675,0.936842,1.473575,1.0,0.023678,5.767108,0.348809,0.114987,0.826603,0.526364
29,(Expensive),(Entire home/apt),0.090232,0.386589,0.081954,0.908257,2.349410,1.0,0.047071,6.686175,0.631327,0.207547,0.850438,0.560124
202,"(One, Expensive)",(Entire home/apt),0.061258,0.386589,0.054636,0.891892,2.307078,1.0,0.030954,5.674048,0.603522,0.138947,0.823759,0.516610
255,"(Average, Some, Cheap)",(Private room),0.082781,0.611755,0.070364,0.850000,1.389445,1.0,0.019722,2.588300,0.305585,0.112732,0.613646,0.482510
254,"(Average, Private room, Some)",(Cheap),0.083609,0.635762,0.070364,0.841584,1.323742,1.0,0.017209,2.299255,0.266879,0.108418,0.565077,0.476131
270,"(Few, Some, Private room)",(Cheap),0.090232,0.635762,0.075331,0.834862,1.313169,1.0,0.017965,2.205666,0.262136,0.115776,0.546622,0.476676
196,"(Some, Private room)",(Cheap),0.268212,0.635762,0.220199,0.820988,1.291345,1.0,0.049680,2.034711,0.308305,0.322034,0.508530,0.583671
147,"(Few, Private room)",(Cheap),0.192053,0.635762,0.154801,0.806034,1.267825,1.0,0.032701,1.877851,0.261462,0.230012,0.467476,0.524762
283,"(None, Cheap, One)",(Private room),0.086921,0.611755,0.069536,0.800000,1.307713,1.0,0.016362,1.941225,0.257706,0.110526,0.484861,0.456834
195,"(Cheap, Some)",(Private room),0.275662,0.611755,0.220199,0.798799,1.305750,1.0,0.051561,1.929636,0.323270,0.330025,0.481767,0.579372
